# Каких пользователей и сколько закачаем?

TLDR:

Из-за ограничений API сэмплируем данные: берём по 1000 пользователй каждой группы

Делаем небольшой offset для к4ачественного сэмплирования

Закачиваем характеристики пользователей (пол и анкету "Любимая музыка", ничего лишнего)

Сохраняем всё в файле big_data.csv

In [1]:
 # coding: utf-8
# наше всё
import numpy as np
import pandas as pd

#настройки pandas, с которыми лучше почти всегда
pd.set_option('display.max_rows', 45000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.max_colwidth', 5000)

import requests
import time

In [2]:
access_token='не_забудь_вставить_сюда_свежий_токен'

In [3]:
# чтобы узнать, как получился этот файл, см. файл bands_findings.ipynb
group_df=pd.read_csv('group_df.csv')

In [4]:
group_df

,id,is_closed,name,photo_100,photo_200,photo_50,screen_name,type
0,6016960,0,Группа Мельница,https://sun1-86.userapi.com/c851036/v851036942/1def16/CVvljudzHL4.jpg?ava=1,https://sun1-28.userapi.com/c851036/v851036942/1def15/KdisGxyT4gw.jpg?ava=1,https://sun1-23.userapi.com/c851036/v851036942/1def17/XjXdWrEpn0Y.jpg?ava=1,melnitsamusic,group
1,59992118,0,Группа «Пикник»,https://sun1-20.userapi.com/c856132/v856132130/c688f/OLCwKy3GnL4.jpg?ava=1,https://sun1-93.userapi.com/c856132/v856132130/c688e/9FltsObk29Q.jpg?ava=1,https://sun1-98.userapi.com/c856132/v856132130/c6890/FoKxnfYG0mM.jpg?ava=1,piknik_info,page
2,7373,0,Король и Шут,https://sun1-97.userapi.com/c849228/v849228903/51561/GWjj17ixQzQ.jpg?ava=1,https://sun1-27.userapi.com/c849228/v849228903/51560/vquG8vNO4IE.jpg?ava=1,https://sun1-24.userapi.com/c849228/v849228903/51562/0AZs2JPNhrY.jpg?ava=1,korol_i_shut_ru,group
3,86583346,0,Диана Арбенина. Ночные Снайперы,https://sun1-14.userapi.com/c847221/v847221611/db787/sI3_fmeHMeQ.jpg?ava=1,https://sun1-84.userapi.com/c847221/v847221611/db786/9WY5qR9UO1Y.jpg?ava=1,https://sun1-26.userapi.com/c847221/v847221611/db788/QkP4KAYDcw0.jpg?ava=1,thenightsnipers,group
4,28057628,0,Сурганова и Оркестр. Светлана Сурганова,https://sun1-29.userapi.com/c851436/v851436406/1e1437/pfDEbVJ3I6k.jpg?ava=1,https://sun1-19.userapi.com/c851436/v851436406/1e1436/L9QTLgLkqdk.jpg?ava=1,https://sun1-94.userapi.com/c851436/v851436406/1e1438/g8RIhXq54us.jpg?ava=1,surganovaofficial,page
5,23408047,0,Земфира,https://sun1-47.userapi.com/c206528/v206528778/c65e8/DFpdL4hxc3M.jpg?ava=1,https://sun1-22.userapi.com/c206528/v206528778/c65e7/rWM5QUPcF9w.jpg?ava=1,https://sun1-98.userapi.com/c206528/v206528778/c65e9/5CpGj47pSWw.jpg?ava=1,zemfira,page
6,173891,0,Мумий Тролль (Mumiy Troll),https://sun1-89.userapi.com/NDMc77B36bA6m0z5PDrkFy72SNK8mqIzdxZisg/4pOI6P_8A8o.jpg?ava=1,https://sun1-27.userapi.com/J4IeMh5g-ep4YeB5_Nj0b164i0JZlP1-bX9kow/lE22W64hwlU.jpg?ava=1,https://sun1-47.userapi.com/PehfZBv9bmEhNeEcJNYK_7GtEVwau_0a40CrOw/NukXnNhTvcA.jpg?ava=1,mumiytrollcom,group
7,225666,0,Би-2,https://sun1-15.userapi.com/c857028/v857028429/181f1f/iSe1RpLf_Nk.jpg?ava=1,https://sun1-47.userapi.com/c857028/v857028429/181f1e/PU86JHXeLEQ.jpg?ava=1,https://sun1-15.userapi.com/c857028/v857028429/181f20/fnWnxX122ow.jpg?ava=1,b2band,page
8,4034,0,Любители групп «Ария» и «Кипелов»,https://sun1-84.userapi.com/EO1es7vO_yaWFwDxoO11xtC8SMpEeRd6wEkKVA/UxnNnZ-rNI4.jpg?ava=1,https://sun1-24.userapi.com/ObesgoE6DY7xmu2ubDnnn9nfIMEdTBCIKrcmJg/7H-kDHzDuhA.jpg?ava=1,https://sun1-17.userapi.com/PbvCKgO_-Ajd_wJ2llcLUID_TS6SFBs2kWfS0w/t881yLcvHUs.jpg?ava=1,club4034,group
9,38465,0,Группа ПилОт,https://sun1-28.userapi.com/c857628/v857628917/17f4c6/Uvv_vsDR0Pk.jpg?ava=1,https://sun1-30.userapi.com/c857628/v857628917/17f4c5/aw2YhAVwQZc.jpg?ava=1,https://sun1-14.userapi.com/c857628/v857628917/17f4c7/qcKAiFqPdWc.jpg?ava=1,pilot_spb_ru,group


In [5]:
# функция делает запрос к API vk.com
# выдаёт список пользователей данной группы (порциями по 1000 штук)
# больше за раз нельзя из-за ограничения API
# на вход vk_id сообщества, отступ в головах и параметр для отладки
def get_group_part(id, offset, verbose):
    url='https://api.vk.com/method/'
    method='groups.getMembers'
    params = (
        ('group_id', id),
        ('v', '5.52'),
        ('offset', offset),
        ('access_token', access_token),
    )
    
    response = requests.get(url+method, params=params)
    
    # если мы включим отладочный параметр, то увидим какую ошибку выдаёт API
    if verbose==1:
        print(response.text[0:3000])
    
    #сразу выкинем мусор из ответа
    return response.json()['response']['items']


# полный список порциями
# в этой реализации не нужен
# на входе только id сообщества
def get_full_user_list(id):
    # идём порциям по 1000 человек (максимум, что даёт API)
    rng=range(0,1000*1000,1000)
    
    full_lst=[]
    for ofs in rng:
        print(ofs)
        try:
            lst=get_group_part(id, ofs, 0)
        except:
            lst=[]
        time.sleep(1)
        full_lst=full_lst+lst
        
    return full_lst

# функция делает запрос к API vk.com
# на выход получает строчку с праметрами сообщества
# выдёт датафрейм с id пользователей, состоящих в сообществе
def get_group_users(df):
    lst=[]
    
    user_df=pd.DataFrame()
    for group in df.values:
        # т.к. процесс долгий, не помешает выводить название группы перед началом
        print(group[2])
        # если упадём по ошибке, запустим ещё раз и узнаем, на что ругается API
        # отсутп 500 - достаточный для рандома для простых смертных
        try:
            lst=get_group_part(group[0], 500, 0)
        except:
            get_group_part(group[0], 500, 1)
            lst=[]
        # здесь задержка в секунду решает все наши проблемы
        time.sleep(1)
        # и сразу сделаем нормальный датафрейм
        user_df_buf=pd.DataFrame(lst, columns=['user_id'])
        user_df_buf['group']=group[2]
        user_df_buf['group_id']=group[0]
        
        user_df=pd.concat([user_df, user_df_buf])
        
    return user_df

In [6]:
# функция делает запрос к API vk.com
# на вход получает id пользователя
# на выходе датафрейм, в которой есть id польователя, 
# имя, фамилия, 
# его пол (1 - женский, 2- мужской, 0 - всё сложно)
# а также то, что пользователь оставил в своей анкете "любимая музыка"
def user_info(id):
    url='https://api.vk.com/method/'
    method='users.get'
    params = (
        ('user_id', id),
        # sex, не gender, но это именно м-ж
        ('fields', 'sex,music'),
        ('v', '5.52'),
        ('access_token', access_token),
    )

    response = requests.get(url+method, params=params)
    try:
        data=pd.io.json.json_normalize(response.json()['response'])
    except:
        print(response.text[0:3000])
        data=pd.DataFrame()
        
    return data

# функция для обработки исключений в анкете
def music_parse(df):
    # может быть так, что поля в ответе просто нет, тогда создадим
    if 'music' not in df.columns:
        df['music']='No info'
            
    # для того, чтобы проще было хранить в csv, заменим ; от эстетов на , 
    df['music']=df['music'].str.replace(';', ',')
    #  а бывает так, что поле есть, но пустое, вставим заглушку, чтобы дальше было проще жить
    df['music']=np.where(df['music']=='','No info', df['music'])
    
    df=df[['first_name', 'id', 'last_name', 'music', 'sex']]
    return df

# основной цикл
# идём по списку пользовтелей и сразу пишем в файл
def get_full_info(df):
    i=0
    for user in df['user_id'].values:
        # получаем инфу о человеке
        tmp_df=user_info(user)
        # обрабатываем её
        tmp_df=music_parse(tmp_df)
        
        # записываем результат
        tmp_df.to_csv('big_data.csv', sep=';', index=False, mode='a+', header=False)
        
        # эмпирически выведеная задержка, которая делает всех счастливыми
        if i%2==0:
            time.sleep(0.6)
        # самодельный прогресс-бар
        if i%100==0:
            print(
                str(
                    round(100*i/len(df),0)
                   )+'%'
                 )
        i=i+1 
            

In [7]:
group_df=group_df[group_df['id'].isin([383, 12093489, 144798458])==True]

In [8]:
group_df

,id,is_closed,name,photo_100,photo_200,photo_50,screen_name,type
16,383,0,Армия АлисА,https://sun1-91.userapi.com/c824409/v824409896/a0b48/MK_lFQQUl3Y.jpg?ava=1,https://sun1-93.userapi.com/c824409/v824409896/a0b47/K8F_HxeIEFA.jpg?ava=1,https://sun1-99.userapi.com/c824409/v824409896/a0b49/IGJJCcNbZ4k.jpg?ava=1,army_alisa,group
17,12093489,0,FLЁUR / ФЛЁР,https://sun9-66.userapi.com/c638427/v638427327/52f0f/oiRZQsg03mI.jpg?ava=1,https://sun9-49.userapi.com/c638427/v638427327/52f0e/Tbgh_o_Lnlw.jpg?ava=1,https://sun9-67.userapi.com/c638427/v638427327/52f10/4duK2VdLhBU.jpg?ava=1,fleur_music_band,group
18,144798458,0,Стас Михайлов,https://sun1-21.userapi.com/c856124/v856124700/1ca48c/7ICJqR39oIA.jpg?ava=1,https://sun1-98.userapi.com/c856124/v856124700/1ca48b/FMqEbDE5dO4.jpg?ava=1,https://sun1-90.userapi.com/c856124/v856124700/1ca48d/w5TxCvLZb6A.jpg?ava=1,mihailov.group,group


In [9]:
user_data=get_group_users(group_df)

Армия АлисА
FLЁUR / ФЛЁР
Стас Михайлов


In [10]:
len(user_data)

3000

In [11]:
# перемешал для того, чтобы по данным можно было делать выводы даже если что-то упадёт
# при сэмплировании не так важно
from sklearn.utils import shuffle
user_data = shuffle(user_data)

In [12]:
user_data.sample(4)

,user_id,group,group_id
780,158948430,Стас Михайлов,144798458
971,339521,Армия АлисА,383
948,12015313,FLЁUR / ФЛЁР,12093489
576,236753,Армия АлисА,383


In [13]:
# сохраняем первый файл
user_data.to_csv('users.csv', index=False)

In [14]:
%%time
# самое тяжёлое и страшное запускается здесь
get_full_info(user_data)

0.0%
3.0%
7.0%
10.0%
13.0%
17.0%
20.0%
23.0%
27.0%
30.0%
33.0%
37.0%
40.0%
43.0%
47.0%
50.0%
53.0%
57.0%
60.0%
63.0%
67.0%
70.0%
73.0%
77.0%
80.0%
83.0%
87.0%
90.0%
93.0%
97.0%
CPU times: user 1min 35s, sys: 3.07 s, total: 1min 38s
Wall time: 19min 48s


In [15]:
# загрузим полученное, посмотрим ему в глаза
user_full_data=pd.read_csv('big_data.csv', header=None, sep=';')
user_full_data.columns=['first_name', 'id', 'last_name', 'music', 'sex']

In [16]:
user_full_data.sample(20)

,first_name,id,last_name,music,sex
37371,Андрей,738555,Долгополов,"Metallica, The Offspring, Nirvana, RHCP, Papa Roach, Limp Bizkit, Cold, Staind, Drowning pool, Sum 41, Blink 182, Наив, Тараканы, linkin park, kiss, hoobastank, Lumen, P.O.D",2
29413,Анастасия,1171112,Малофеева,rus rock,1
31790,Екатерина,130349,Хмелевская,No info,1
24388,Алёна,708341,Лещинская,No info,1
24466,Любовь,998052,Ли,No info,1
40801,Андрей,182367,Осипов,No info,2
29941,Ольга,334497,Каплан,the Doors,1
32841,Мария,70412444,Апкаликова,No info,1
25804,Павел,323386,Данилов,No info,2
26222,Алеся,942228,Шляпникова,No info,1
